In [1]:
import numpy as np
import os
import cv2
import keras
import sklearn
import pandas
from time import time
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.utils import to_categorical
from keras.models import load_model
from keras.layers import *
from keras import layers
from keras import Model
from keras import optimizers
from keras import regularizers
from keras.callbacks import TensorBoard
from keras import optimizers
import matplotlib.pyplot as plt
from keras.applications import *
from sklearn.metrics import classification_report
import time

In [2]:
IMG_SIZE = 224
DATA_PATH = 'F:/Machine_Learning/Projects/paper_implementation/Paper_2_GoogleNet/Hockey_fights/Dataframes'

In [3]:
violent_frames = []
non_violent_frames = []
c = 0
for frames in os.listdir(DATA_PATH):
    frame = cv2.resize(cv2.imread(os.path.join(DATA_PATH,frames)), (IMG_SIZE, IMG_SIZE))

    if c < 20057:
        violent_frames.append(frame)
    elif c >= 20057:
        non_violent_frames.append(frame)
    c += 1    

np.save('hockey_data_for_raw_model_violent.npy', violent_frames)
np.save('hockey_data_for_raw_model_nonviolent.npy', non_violent_frames)

In [2]:
violent_frames = np.load('hockey_data_for_raw_model_violent.npy', allow_pickle= True)
non_violent_frames = np.load('hockey_data_for_raw_model_nonviolent.npy', allow_pickle= True)

In [3]:
print(len(violent_frames))
print(len(non_violent_frames))

20057
19999


In [4]:
print("+++ Extracting feature... +++")
    
violent_frames = np.asarray(violent_frames)
non_violent_frames = np.asarray(non_violent_frames)

print ("Before Feature extraction: ")
print(violent_frames.shape,non_violent_frames.shape)
# all_data = np.vstack((violent_frames,non_violent_frames))
# print("Adding all data: ", all_data.shape)

+++ Extracting feature... +++
Before Feature extraction: 
(20057, 224, 224, 3) (19999, 224, 224, 3)


In [5]:
loaded_model = resnet50.ResNet50(input_shape=(224,224,3), include_top=False)
loaded_model = Model(loaded_model.input,loaded_model.output)

In [6]:
with tf.device('/cpu'):
    features_violent = loaded_model.predict(violent_frames,batch_size=10,verbose=1)
np.save('resnet__hockey_feature_violent.npy', np.asarray(features_violent))

2006/2006 [==============================] - 880s 439ms/step


In [7]:
with tf.device('/cpu'):
    features_nonviolent = loaded_model.predict(non_violent_frames,batch_size=10,verbose=1)
np.save('resnet__hockey_feature_nonviolent.npy', np.asarray(features_nonviolent))

2000/2000 [==============================] - 863s 431ms/step


In [2]:
features_violent = np.load('resnet__hockey_feature_violent.npy', allow_pickle = True)
features_nonviolent = np.load('resnet__hockey_feature_nonviolent.npy', allow_pickle = True)

In [3]:
violent_features = features_violent[0:19980]
non_violent_features = features_nonviolent[-19980:]
print("Violent features: ", violent_features.shape)
print("Non Violent features: ", non_violent_features.shape)

Violent features:  (19980, 7, 7, 2048)
Non Violent features:  (19980, 7, 7, 2048)


In [4]:
violent_vid = []
non_violent_vid = []

i = 0
while i < len(violent_features):
    violent_vid.append(np.array(violent_features[i:i+30]))
    i = i+30

i = 0
while i < len(non_violent_features):
    non_violent_vid.append(np.array(non_violent_features[i:i+30]))
    i = i+30

violent_vid = np.asarray(violent_vid)
non_violent_vid = np.asarray(non_violent_vid)

In [5]:
violent_y, non_violent_y = np.zeros(len(violent_vid)), np.ones(len(non_violent_vid))

In [6]:
#Special Case for this paper only. 
#Violent = 0
#Non violent = 1

print("Violent Video Seq: ", violent_vid.shape,"Non_violent video Seq: ", non_violent_vid.shape)
print("Violent Label: ", violent_y.shape, "Non_violent Label: ", non_violent_y.shape)

Violent Video Seq:  (666, 30, 7, 7, 2048) Non_violent video Seq:  (666, 30, 7, 7, 2048)
Violent Label:  (666,) Non_violent Label:  (666,)


In [7]:
X = np.vstack((violent_vid,non_violent_vid))
y = np.append(violent_y, non_violent_y)
X = np.reshape(X,(X.shape[0],X.shape[1],X.shape[2]*X.shape[3]*X.shape[4]))
print(X.shape)
print(y.shape)

(1332, 30, 100352)
(1332,)


In [8]:
model = load_model('./resnet_LSTM_model.h5')
model.load_weights('./resnet_LSTM_weights.h5')
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cu_dnnlstm (CuDNNLSTM)       (None, 50)                20080800  
_________________________________________________________________
dense (Dense)                (None, 1)                 51        
Total params: 20,080,851
Trainable params: 20,080,851
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.evaluate(x = X, y =y, batch_size= 5, verbose = 1)

267/267 [==============================] - 31s 36ms/step - loss: 1.1996 - accuracy: 0.5278


[1.1996160745620728, 0.5277777910232544]